# XGBoost modeli

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv(r"train.csv")

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data["satisfaction"] = le.fit_transform(data["satisfaction"])
data["Customer Type"] = le.fit_transform(data["Customer Type"])



data["Class"] = data["Class"].map({ "Eco":1,"Eco Plus":2,"Business":3})


"""data["JobType"] = data["JobType"].map({" ?":np.nan, " Never-worked":0,
                                       " Without-pay":1," Self-emp-not-inc":2,
                                       " Self-emp-inc":3," Private":4,
                                         "Local-gov":5," State-gov":6," Federal-gov":7," Without-pay":8})"""




Y = data.iloc[:,-1]
X = data[["Class", "Online boarding","Inflight entertainment"]]

from sklearn.impute import SimpleImputer
imputer  = SimpleImputer(missing_values = np.nan, strategy = "mean")
missing_data = X.values
missing_data = missing_data.reshape(len(missing_data),X.shape[1])
imputer = imputer.fit(missing_data)
X = imputer.transform(missing_data)
X1 = pd.DataFrame(X, columns = ["Class", "Online boarding","Inflight entertainment"])


from sklearn.model_selection import train_test_split


x_train, x_test,y_train,y_test = train_test_split(X1,Y,test_size=0.33, random_state=0)

x_train = x_train.sort_index().values
y_train = y_train.sort_index().values
x_test = x_test.sort_index().values
y_test = y_test.sort_index().values

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

x_train = sc.fit_transform(x_train)

x_test = sc.transform(x_test)


In [19]:
x_train1, x_test1 = x_train, x_test
y_train1, y_test1 = y_train, y_test

In [20]:
from xgboost import XGBClassifier

xgb = XGBClassifier(eval_metric='mlogloss',use_label_encoder=False)
xgb.fit(x_train,y_train)
y_pred = xgb.predict(x_test)



In [21]:
from sklearn.metrics import confusion_matrix

cm1 = confusion_matrix(y_test,y_pred)

cm1

array([[16989,  2432],
       [ 3007, 11861]], dtype=int64)

In [22]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
def print_metrics(labels, preds):
    print("Precision Score: {}".format(precision_score(labels, preds, average = 'macro')))
    print("Recall Score: {}".format(recall_score(labels, preds, average = 'macro')))
    print("Accuracy Score: {}".format(accuracy_score(labels, preds)))
    print("F1 Score: {}".format(f1_score(labels, preds, average = 'macro')))
    
print_metrics(y_test, y_pred) 

Precision Score: 0.839733351063972
Recall Score: 0.8362641465447576
Accuracy Score: 0.8413777013036251
F1 Score: 0.8377488072254495


In [23]:


from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV, cross_val_score
crossvalidation=KFold(n_splits=3,shuffle=True,random_state=1)

"""for depth in range (1,10):
    tree_classifier=DecisionTreeClassifier(max_depth=depth,random_state=1)
    if tree_classifier.fit(x_train1,y_train1).max_depth<depth:
        break
    score=np.mean(cross_val_score(tree_classifier,x_train,y_train,scoring='accuracy', cv=crossvalidation,n_jobs=-1))
    print(depth, score)"""

"for depth in range (1,10):\n    tree_classifier=DecisionTreeClassifier(max_depth=depth,random_state=1)\n    if tree_classifier.fit(x_train1,y_train1).max_depth<depth:\n        break\n    score=np.mean(cross_val_score(tree_classifier,x_train,y_train,scoring='accuracy', cv=crossvalidation,n_jobs=-1))\n    print(depth, score)"

In [24]:
import time
start = time.time()



parameters = {
    'max_depth': range (10, 30, 10),
    'n_estimators': range(50, 200, 50),
    'learning_rate': [0.01, 0.1,1.0]}

grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=parameters,
    scoring = 'accuracy',
    n_jobs = -1,
    cv = 10,
    verbose=True)

grid_search.fit(x_train, y_train)

print("best param: ", grid_search.best_params_)

end = time.time()

print("xgbosst GridSerach çalışma süresi", end-start)


Fitting 10 folds for each of 18 candidates, totalling 180 fits
best param:  {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 100}
xgbosst GridSerach çalışma süresi 1816.723955631256


In [25]:
grid_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='mlogloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=10, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              use_label_encoder=False, validate_parameters=1, verbosity=None)

In [26]:
grid_search.best_score_

0.8438268741580343

In [10]:
import time
from optuna import Trial, visualization
import optuna
start = time.time()

def tune(objective):
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=20)

    params = study.best_params
    best_score = study.best_value
    print(f"Best score: {best_score}\n")
    print(f"Optimized parameters: {params}\n")
    return params


def xgboost_objective(trial):
    maxdepth_number  = trial.suggest_int("max_depth", 10, 30,10)
    estimators_number = trial.suggest_int("n_estimators", 50, 200,50)
    learn_rate = trial.suggest_categorical("learning_rate", [0.01, 0.1,1.0 ])


    xgb = XGBClassifier(n_estimators = estimators_number,
                        max_depth = maxdepth_number,
                        learning_rate = learn_rate,eval_metric='mlogloss',use_label_encoder=False,n_jobs=-1)
    
    scores = cross_val_score(xgb,x_train,y_train,cv=5, scoring = 'accuracy', n_jobs = -1)
    
    return scores.mean()

xgboost_params = tune(xgboost_objective)

print(xgboost_params)

end = time.time()


print("xgbosst Optuna çalışma süresi " + str(end-start) + " sn")
    

[I 2021-09-21 18:18:39,648] A new study created in memory with name: no-name-1edd36a0-4b3e-4bae-b5ca-077595c3e9ed
[I 2021-09-21 18:18:41,951] Trial 0 finished with value: 0.8433958198664081 and parameters: {'max_depth': 30, 'n_estimators': 50, 'learning_rate': 0.01}. Best is trial 0 with value: 0.8433958198664081.
[I 2021-09-21 18:18:43,281] Trial 1 finished with value: 0.8434820081878905 and parameters: {'max_depth': 10, 'n_estimators': 100, 'learning_rate': 1.0}. Best is trial 1 with value: 0.8434820081878905.
[I 2021-09-21 18:18:49,747] Trial 2 finished with value: 0.8434820081878905 and parameters: {'max_depth': 10, 'n_estimators': 200, 'learning_rate': 0.1}. Best is trial 1 with value: 0.8434820081878905.
[I 2021-09-21 18:18:54,571] Trial 3 finished with value: 0.8434820081878905 and parameters: {'max_depth': 20, 'n_estimators': 100, 'learning_rate': 0.1}. Best is trial 1 with value: 0.8434820081878905.
[I 2021-09-21 18:18:57,096] Trial 4 finished with value: 0.8434820081878905 an

Best score: 0.8434820081878905

Optimized parameters: {'max_depth': 10, 'n_estimators': 100, 'learning_rate': 1.0}

{'max_depth': 10, 'n_estimators': 100, 'learning_rate': 1.0}
xgbosst Optuna çalışma süresi 72.71788692474365 sn


# AdaBoost modeli

In [11]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

abc = AdaBoostClassifier(DecisionTreeClassifier(max_depth=50), n_estimators=10)
abc.fit(x_train1, y_train1)
y_pred1 = abc.predict(x_test1)


In [12]:
from sklearn.metrics import confusion_matrix

cm2 = confusion_matrix(y_test1,y_pred1)

cm2

array([[16989,  2432],
       [ 3007, 11861]], dtype=int64)

In [13]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
def print_metrics(labels, preds):
    print("Precision Score: {}".format(precision_score(labels, preds, average = 'macro')))
    print("Recall Score: {}".format(recall_score(labels, preds, average = 'macro')))
    print("Accuracy Score: {}".format(accuracy_score(labels, preds)))
    print("F1 Score: {}".format(f1_score(labels, preds, average = 'macro')))
    
print_metrics(y_test1, y_pred1) 

Precision Score: 0.839733351063972
Recall Score: 0.8362641465447576
Accuracy Score: 0.8413777013036251
F1 Score: 0.8377488072254495


In [14]:
from sklearn.model_selection import GridSearchCV, cross_val_score

cls = cross_val_score(AdaBoostClassifier(DecisionTreeClassifier(max_depth=5), n_estimators=10), 
                              x_train1, y_train1, cv=5)

cls, np.mean(cls)

(array([0.84313725, 0.84486102, 0.83861237, 0.84622567, 0.84471737]),
 0.8435107376283847)

In [15]:

from sklearn.model_selection import KFold

crossvalidation=KFold(n_splits=10,shuffle=True,random_state=1)

for depth in range (1,10):
    tree_classifier=DecisionTreeClassifier(max_depth=depth,random_state=1)
    if tree_classifier.fit(x_train1,y_train1).max_depth<depth:
        break
    score=np.mean(cross_val_score(tree_classifier,x_train1,y_train1,scoring='accuracy', cv=crossvalidation,n_jobs=1))
    print(depth, score)

1 0.786741696391035
2 0.8115349292262828
3 0.8278100613996617
4 0.835121740153548
5 0.8438986306974539
6 0.8439272982345395
7 0.8436974627094006
8 0.8436974627094006
9 0.8436974627094006


In [16]:
import time

start = time.time()


ada=AdaBoostClassifier()
search_grid={'n_estimators': [50,100,200],'learning_rate':[0.001,0.01,0.1]}

search=GridSearchCV(estimator=ada ,param_grid=search_grid, scoring='accuracy',cv=crossvalidation)
search.fit(x_train1,y_train1)

print("best parametre: ", search.best_params_)
print("best score: ", search.best_score_)

end = time.time()

print("adaboost GridSearch çalışma süresi: ", end-start)

best parametre:  {'learning_rate': 0.1, 'n_estimators': 200}
best score:  0.8353229576644464
adaboost GridSearch çalışma süresi:  209.24717831611633


In [17]:
from optuna import Trial, visualization
import optuna
start = time.time()

def tune(objective):
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=20)

    params = study.best_params
    best_score = study.best_value
    print(f"Best score: {best_score}\n")
    print(f"Optimized parameters: {params}\n")
    return params


def adaboost_objective(trial):
    
    n_estimator1 = trial.suggest_categorical("n_estimators", [50, 100,200])
    learn_rate1 = trial.suggest_categorical("learning_rate", [0.01, 0.1,1.0 ])


    ada=AdaBoostClassifier(n_estimators = n_estimator1, learning_rate = learn_rate1)
    
    scores = cross_val_score(ada,x_train,y_train,cv=3, scoring = 'accuracy', n_jobs = -1)
    
    return scores.mean()

adabosst_params = tune(adaboost_objective)

print(xgboost_params)

end = time.time()


print("adaboost Optuna çalışma süresi " + str(end-start) + " sn")
    

[I 2021-09-21 18:23:25,568] A new study created in memory with name: no-name-0ba35be9-55a6-4659-aefa-7728af41f973
[I 2021-09-21 18:23:26,977] Trial 0 finished with value: 0.8346477052359406 and parameters: {'n_estimators': 50, 'learning_rate': 1.0}. Best is trial 0 with value: 0.8346477052359406.
[I 2021-09-21 18:23:28,695] Trial 1 finished with value: 0.8346477052359406 and parameters: {'n_estimators': 50, 'learning_rate': 1.0}. Best is trial 0 with value: 0.8346477052359406.
[I 2021-09-21 18:23:33,435] Trial 2 finished with value: 0.8237879767291533 and parameters: {'n_estimators': 200, 'learning_rate': 0.01}. Best is trial 0 with value: 0.8346477052359406.
[I 2021-09-21 18:23:37,987] Trial 3 finished with value: 0.8346477052359406 and parameters: {'n_estimators': 200, 'learning_rate': 1.0}. Best is trial 0 with value: 0.8346477052359406.
[I 2021-09-21 18:23:40,328] Trial 4 finished with value: 0.8346477052359406 and parameters: {'n_estimators': 100, 'learning_rate': 1.0}. Best is tr

Best score: 0.835523953171012

Optimized parameters: {'n_estimators': 200, 'learning_rate': 0.1}

{'max_depth': 10, 'n_estimators': 100, 'learning_rate': 1.0}
adaboost Optuna çalışma süresi 71.1243097782135 sn
